In [23]:
# Lib imports
import pandas as pd

In [6]:
import boto3
s3 = boto3.resource('s3')

# Print out bucket names
listy = []
for bucket in s3.buckets.all():
    listy.append(bucket.name)

In [8]:
listy[1]

'kagstatefarm'

In [18]:
# List everything in bucket
statefarm = s3.Bucket('kagstatefarm')
statefarm_files = []
for file_object in statefarm.objects.all():
    statefarm_files.append(file_object.key)

In [21]:
split_keys = [ [entry.split('/')[0], entry] for entry in statefarm_files]

In [24]:
split_keys_frame =  pd.DataFrame(split_keys, columns = ['folder', 's3_key'])

In [26]:
train_set = split_keys_frame[split_keys_frame.folder == 'train']
test_set = split_keys_frame[split_keys_frame.folder == 'test']

In [29]:
test_set

,folder,s3_key
0,test,test/img_1.jpg
1,test,test/img_10.jpg
2,test,test/img_100.jpg
3,test,test/img_1000.jpg
4,test,test/img_100000.jpg
5,test,test/img_100001.jpg
6,test,test/img_100002.jpg
7,test,test/img_100003.jpg
8,test,test/img_100004.jpg
9,test,test/img_100005.jpg
